In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## PAPER : 
## Real-world Anomaly Detection in Surveillance Videos
https://arxiv.org/pdf/1801.04264v3.pdf



In [2]:
import numpy as np
import os
import cv2
import sklearn.preprocessing
from os import listdir, mkdir
from os.path import isfile, join, isdir

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.models import model_from_json

import sklearn.preprocessing
import scipy.io as sio
import sklearn.metrics

import pandas as pd
import matplotlib.pyplot as plt
import keras.optimizers
import scipy.io

from datetime import datetime


###  Configurations

In [3]:
frame_height = 240
frame_width = 320
channels = 3
frame_count = 16
features_per_bag = 32
normal_videos_path='C:/Users/Administrator/Desktop/GP Project/crimeucfdataset/Dataset/train/normal'
abnormal_videos_path=r"C:\Users\Administrator\Desktop\GP V2 Project\Dataset\Abnormal\Abuse"
test_set=r"C:\Users\Administrator\Desktop\GP Project\Test_split\test"

C3D_MEAN_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/c3d_mean.npy'

raw_normal_train_features="C:/Users/Administrator/Desktop/GP Project/crimeucfdataset/Dataset/raw_c3d_features/raw_normal_train_features"
raw_abnormal_train_features='C:/Users/Administrator/Desktop/GP Project/crimeucfdataset/Dataset/raw_c3d_features/raw_abnormal_train_features'
raw_test_features='C:/Users/Administrator/Desktop/GP Project/crimeucfdataset/Dataset/raw_c3d_features/raw_test_features'

processed_normal_train_features="C:/Users/Administrator/Desktop/GP Project/crimeucfdataset/Dataset/processed_c3d_features/processed_normal_train_features"
processed_abnormal_train_features='C:/Users/Administrator/Desktop/GP Project/crimeucfdataset/Dataset/processed_c3d_features/processed_abnormal_train_features'
processed_test_features="C:/Users/Administrator/Desktop/GP Project/crimeucfdataset/Dataset/processed_c3d_features/processed_test_features"

trained_models=r"C:\Users\Administrator\Desktop\GP Project\trained_models_final"
preds_folder=r"C:\Users\Administrator\Desktop\GP Project\Test_split\preds_folder"
test_temporal_annotations=r"C:\Users\Administrator\Desktop\GP Project\Test_split\temporal-annotation.txt"

### Prepare Dataset

In [ ]:
processed_anomaly=r"C:\Users\Administrator\Desktop\GP Project\crimeucfdataset\Dataset\processed_c3d_features\processed_abnormal_train_features"

file_name_list = []
file_path_list = []
file_type_list = []
file_class_list = []
file_npy_path_list = []

classes = {"Normal":0, "Abbuse":1,"Arrest":2,"Arson":3,"Assault":4,"Explosion":6, 'Burglary':5, 'Fighting':7,"RoadAccidents":8
          ,"Robbery":9,"Shooting":10,"ShopLifting":11,"Stealing":12,"Vandalism":13}
types = {"Normal":0, "Abnormal":1}
def build_dataset_df():
    global file_name_list 
    global file_path_list
    global file_type_list 
    global file_class_list 
    global file_npy_path_list

    file_name_list.clear()
    file_path_list.clear()
    file_type_list.clear()
    file_class_list.clear()
    file_npy_path_list.clear()


    for root, subdirs, files in os.walk(processed_anomaly):
        for filename in files:
            

            file_path = os.path.join(root, filename)
            file_name = filename.split('.')[0]
            
            
            
            file_name_list.append(file_name)
            file_path_list.append(file_path)
            file_class = file_path.split('\\')[-2]
            #print(file_class)
            if file_class in classes.keys():
                file_type_list.append(types["Abnormal"])
                file_class_list.append(classes[file_class])
            else:
                file_type_list.append(types["Normal"])
                file_class_list.append(classes["Normal"])

            #npy_path = Save2Npy(file_path, file_name, Config.save_npy_dir)
            #file_npy_path_list.append(npy_path)
            

In [ ]:
build_dataset_df()
dataset_df = pd.DataFrame(list(zip(file_name_list, file_path_list, file_type_list, file_class_list)),
                          columns =['file_name', 'file_path', 'file_type', 'file_class'])
print(len(file_path_list), len(file_name_list), len(file_type_list), len(file_class_list))
dataset_df.head()


In [ ]:
dataset_df['file_class'].value_counts()

In [4]:

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Data Prerocessing 

In [5]:
def sliding_window(arr, size, stride):
    """Apply sliding window to an array, getting chunks of
    of specified size using the specified stride
    :param arr: Array to be divided
    :param size: Size of the chunks
    :param stride: Number of frames to skip for the next chunk
    :returns: Tensor with the resulting chunks
    :rtype: np.ndarray
    """
    num_chunks = int((len(arr) - size) / stride) + 2
    result = []
    for i in range(0,  num_chunks * stride, stride):
        if len(arr[i:i + size]) > 0:
            result.append(arr[i:i + size])
    return np.array(result)


def interpolate(features, features_per_bag):
    """Transform a bag with an arbitrary number of features into a bag
    with a fixed amount, using interpolation of consecutive features
    :param features: Bag of features to pad
    :param features_per_bag: Number of features to obtain
    :returns: Interpolated features
    :rtype: np.ndarray
    """
    feature_size = np.array(features).shape[1]
    interpolated_features = np.zeros((features_per_bag, feature_size))
    interpolation_indices = np.round(np.linspace(0, len(features) - 1, num=features_per_bag + 1))
    count = 0
    for index in range(0, len(interpolation_indices)-1):
        start = int(interpolation_indices[index])
        end = int(interpolation_indices[index + 1])

        assert end >= start

        if start == end:
            temp_vect = features[start, :]
        else:
            temp_vect = np.mean(features[start:end+1, :], axis=0)

        temp_vect = temp_vect / np.linalg.norm(temp_vect)

        if np.linalg.norm(temp_vect) == 0:
            print("Error")

        interpolated_features[count,:]=temp_vect
        count = count + 1

    return np.array(interpolated_features)


def extrapolate(outputs, num_frames):
    """Expand output to match the video length
    :param outputs: Array of predicted outputs
    :param num_frames: Expected size of the output array
    :returns: Array of output size
    :rtype: np.ndarray
    """

    extrapolated_outputs = []
    extrapolation_indices = np.round(np.linspace(0, len(outputs) - 1, num=num_frames))
    for index in extrapolation_indices:
        extrapolated_outputs.append(outputs[int(index)])
    return np.array(extrapolated_outputs)

In [6]:
def get_video_clips(video_path):
    """Divides the input video into non-overlapping clips
    :param video_path: Path to the video
    :returns: Array with the fragments of video
    :rtype: np.ndarray
    """
    frames = get_video_frames(video_path)
    clips = sliding_window(frames, frame_count, frame_count)
    return clips, len(frames)


def get_video_frames(video_path):
    """Reads the video given a file path
    :param video_path: Path to the video
    :returns: Video as an array of frames
    :rtype: np.ndarray
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        else:
            break
    cap.release()
    return frames

In [8]:

def preprocess_input(video):
    """Preprocess video input to make it suitable for feature extraction.
    The video is resized, cropped, resampled and training mean is substracted
    to make it suitable for the network
    :param video: Video to be processed
    :returns: Preprocessed video
    :rtype: np.ndarray
    """

    intervals = np.ceil(np.linspace(0, video.shape[0] - 1, 16)).astype(int)
    frames = video[intervals]

    # Reshape to 128x171
    reshape_frames = np.zeros((frames.shape[0], 128, 171, frames.shape[3]))
    for i, img in enumerate(frames):
        img = cv2.resize(img, (171, 128), cv2.INTER_CUBIC)
        reshape_frames[i, :, :, :] = img

    mean_path = tf.keras.utils.get_file('c3d_mean.npy',
                         C3D_MEAN_PATH,
                         cache_subdir='models',
                         md5_hash='08a07d9761e76097985124d9e8b2fe34')

    mean = np.load(mean_path)
    reshape_frames -= mean
    # Crop to 112x112
    reshape_frames = reshape_frames[:, 8:120, 30:142, :]
    # Add extra dimension for samples
    reshape_frames = np.expand_dims(reshape_frames, axis=0)

    return reshape_frames


def C3D(weights='sports1M'):
    """Creation of the full C3D architecture
    :param weights: Weights to be loaded into the network. If None,
    the network is randomly initialized.
    :returns: Network model
    :rtype: keras.model
    """

    if weights not in {'sports1M', None}:
        raise ValueError('weights should be either be sports1M or None')

    if tf.keras.backend.image_data_format() == 'channels_last':
        shape = (16, 112, 112, 3)
    else:
        shape = (3, 16, 112, 112)

    model = Sequential()
    model.add(
        Conv3D(64,3,activation='relu',padding='same',name='conv1', input_shape=shape))
    model.add( MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),padding='same', name='pool1'))

    model.add(Conv3D(128, 3, activation='relu', padding='same', name='conv2'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2),strides=(2, 2, 2),padding='valid',name='pool2'))

    model.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3a'))
    model.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3b'))
    model.add(
        MaxPooling3D(pool_size=(2, 2, 2),
                     strides=(2, 2, 2),
                     padding='valid',
                     name='pool3'))

    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4a'))
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4b'))
    model.add(
        MaxPooling3D(pool_size=(2, 2, 2),
                     strides=(2, 2, 2),
                     padding='valid',
                     name='pool4'))

    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5a'))
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5b'))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(
        MaxPooling3D(pool_size=(2, 2, 2),
                     strides=(2, 2, 2),
                     padding='valid',
                     name='pool5'))

    model.add(Flatten())

    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(0.5))
    model.add(Dense(487, activation='softmax', name='fc8'))

    if weights == 'sports1M':
        model.load_weights('C:/Users/Administrator/Desktop/GP Project/c3d_sports1m.h5')

    return model


def c3d_feature_extractor():
    """Creation of the feature extraction architecture. This network is
    formed by a subset of the original C3D architecture (from the
    beginning to fc6 layer)
    :returns: Feature extraction model
    :rtype: keras.model
    """
    model = C3D(weights='sports1M')
    layer_name = 'fc6'
    feature_extractor_model = Model(inputs=model.input,
                                    outputs=model.get_layer(layer_name).output)
    return feature_extractor_model

## Training: Feature Extraction 

### Normal Videos

In [ ]:
#Raw Normal Train _ features 
feature_extractor = c3d_feature_extractor()
normal_videos = os.listdir(normal_videos_path)
normal_videos.sort()

print("Processing normal videos...")
for vid_name in normal_videos:
    print("Processing {}".format(vid_name))
    vid_path = os.path.join(normal_videos_path, vid_name)
    feats_path = os.path.join(
        raw_normal_train_features, vid_name + ".npy"
    )

    clips, frames = get_video_clips(vid_path)

    # Remove last clip if number of frames is not equal to 16
    if frames % 16 != 0:
        clips = clips[:-1]

    prep_clips = [preprocess_input(np.array(clip)) for clip in clips]
    prep_clips = np.vstack(prep_clips)

    features = feature_extractor.predict(prep_clips)
    features = sklearn.preprocessing.normalize(features, axis=1)

    with open(feats_path, "wb") as f:
        np.save(f, features)

### Abnormal Videos

In [ ]:
#Abnormal Videos:
feature_extractor = c3d_feature_extractor()
abnormal_videos = os.listdir(abnormal_videos_path)
abnormal_videos.sort()
print("Processing abnormal videos...")
for vid_name in abnormal_videos:
    print("Processing {}".format(vid_name))
    vid_path = os.path.join(abnormal_videos_path, vid_name)
    feats_path = os.path.join(
       raw_abnormal_train_features, vid_name + ".npy"
    )

    clips, frames = get_video_clips(vid_path)

    # Remove last clip if number of frames is not equal to 16
    if frames % 16 != 0:
        clips = clips[:-1]

    prep_clips = [preprocess_input(np.array(clip)) for clip in clips]
    prep_clips = np.vstack(prep_clips)

    features = feature_extractor.predict(prep_clips)
    features = sklearn.preprocessing.normalize(features, axis=1)

    with open(feats_path, "wb") as f:
        np.save(f, features)

### Test Videos

In [ ]:
#Test set
feature_extractor = c3d_feature_extractor()
test_videos = os.listdir(test_set)
test_videos.sort()
print("Processing test videos...")
for vid_name in test_videos:
    print("Processing {}".format(vid_name))
    vid_path = os.path.join(test_set, vid_name)
    feats_path = os.path.join(raw_test_features, vid_name[:-9] + ".npy")

    clips, frames = get_video_clips(vid_path)

    # Remove last clip if number of frames is not equal to 16
    if frames % 16 != 0:
        clips = clips[:-1]

    prep_clips = [preprocess_input(np.array(clip)) for clip in clips]
    prep_clips = np.vstack(prep_clips)

    features = feature_extractor.predict(prep_clips)
    features = sklearn.preprocessing.normalize(features, axis=1)

    with open(feats_path, "wb") as f:
        np.save(f, features)

### Preprocessing : Transform videos into segments

In [ ]:
def transform_into_segments(features, n_segments=32):
    if features.shape[0] < n_segments:
        raise RuntimeError(
            "Number of prev segments lesser than expected output size"
        )

    cuts = np.linspace(0, features.shape[0], n_segments,
                       dtype=int, endpoint=False)

    new_feats = []
    for i, j in zip(cuts[:-1], cuts[1:]):
        new_feats.append(np.mean(features[i:j,:], axis=0))

    new_feats.append(np.mean(features[cuts[-1]:,:], axis=0))

    new_feats = np.array(new_feats)
    new_feats = sklearn.preprocessing.normalize(new_feats, axis=1)
    return new_feats

# for filename in os.listdir(raw_normal_train_features):
#     print("Processing {}".format(filename))
#     raw_file_path = os.path.join(
#         raw_normal_train_features, filename
#     )
#     processed_file_path = os.path.join(
#         processed_normal_train_features, filename
#     )

#     with open(raw_file_path, "rb") as f:
#         feats = np.load(f, allow_pickle=True)

#     try:
#         new_feats = transform_into_segments(feats)
#         with open(processed_file_path, "wb") as f:
#             np.save(f, new_feats, allow_pickle=True)
#     except RuntimeError:
#         print("Video {} too short".format(filename))

# for filename in os.listdir(raw_abnormal_train_features):
#     print("Processing {}".format(filename))
#     raw_file_path = os.path.join(
#         raw_abnormal_train_features, filename
#     )
#     processed_file_path = os.path.join(
#         processed_abnormal_train_features, filename
#     )
#     with open(raw_file_path, "rb") as f:
#         feats = np.load(f, allow_pickle=True)

#     try:
#         new_feats = transform_into_segments(feats)
#         with open(processed_file_path, "wb") as f:
#             np.save(f, new_feats, allow_pickle=True)
#     except RuntimeError:
#         print("Video {} too short".format(filename))

for filename in os.listdir(raw_test_features):
    print("Processing {}".format(filename))
    raw_file_path = os.path.join(
        raw_test_features, filename
    )
    processed_file_path = os.path.join(
        processed_test_features, filename
    )
    with open(raw_file_path, "rb") as f:
        feats = np.load(f, allow_pickle=True)

    try:
        new_feats = transform_into_segments(feats)
        with open(processed_file_path, "wb") as f:
            np.save(f, new_feats, allow_pickle=True)
    except RuntimeError:
        print("Video {} too short".format(filename))

## Build Classifier Model

In [10]:


def classifier_model():
    """Build the classifier
    :returns: Classifier model
    :rtype: keras.Model
    """
    model = Sequential()
    model.add(Dense(512, input_dim=4096, kernel_initializer='glorot_normal',
                    kernel_regularizer=l2(0.001), activation='relu'))
    model.add(Dropout(0.6))
    
    model.add(Dense(256, kernel_initializer='glorot_normal',
                    kernel_regularizer=l2(0.001)))
    model.add(Dense(32, kernel_initializer='glorot_normal',
                    kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.6))
    model.add(Dense(1, kernel_initializer='glorot_normal',
                    kernel_regularizer=l2(0.001), activation='sigmoid'))
    return model

# Original Model :
def classifier_model_o():
    """Build the classifier
    :returns: Classifier model
    :rtype: keras.Model
    """
    model = Sequential()
    model.add(Dense(512, input_dim=4096, kernel_initializer='glorot_normal',
                    kernel_regularizer=l2(0.001), activation='relu'))
    model.add(Dropout(0.6))
    
    model.add(Dense(32, kernel_initializer='glorot_normal',
                    kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.6))
    model.add(Dense(1, kernel_initializer='glorot_normal',
                    kernel_regularizer=l2(0.001), activation='sigmoid'))
    return model

def build_classifier_model():
    """Build the classifier and load the pretrained weights
    :returns:
    :rtype:
    """
    model = classifier_model()
    model = load_weights(model, r"C:\Users\Administrator\Desktop\GP Project\trained_models_final\weights.mat")
    return model


def build_classifier_model_o():
    """Build the classifier and load the pretrained weights
    :returns:
    :rtype:
    """
    model = classifier_model_o()
    model = load_weights(model, 'C:/Users/Administrator/Desktop/GP Project/weights_L1L2.mat')
    return model

def conv_dict(dict2):
    """Prepare the dictionary of weights to be loaded by the network
    :param dict2: Dictionary to format
    :returns: The dictionary properly formatted
    :rtype: dict
    """
    dict = {}
    for i in range(len(dict2)):
        if str(i) in dict2:
            if dict2[str(i)].shape == (0, 0):
                dict[str(i)] = dict2[str(i)]
            else:
                weights = dict2[str(i)][0]
                weights2 = []
                for weight in weights:
                    if weight.shape in [(1, x) for x in range(0, 5000)]:
                        weights2.append(weight[0])
                    else:
                        weights2.append(weight)
                dict[str(i)] = weights2
    return dict


def load_weights(model, weights_file):
    """Loads the pretrained weights into the network architecture
    :param model: keras model of the network
    :param weights_file: Path to the weights file
    :returns: The input model with the weights properly loaded
    :rtype: keras.model
    """
    dict2 = sio.loadmat(weights_file)
    dict = conv_dict(dict2)
    i = 0
    for layer in model.layers:
        weights = dict[str(i)]
        layer.set_weights(weights)
        i += 1
    return model

if __name__ == '__main__':
    model = build_classifier_model()
    #model=classifier_model()
    model.summary()
    

ResourceExhaustedError: OOM when allocating tensor with shape[4096,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:TruncatedNormal]

## Train Classifier 

In [ ]:

processed_normal_train_features="C:/Users/Administrator/Desktop/GP Project/crimeucfdataset/Dataset/processed_c3d_features/processed_normal_train_features"

processed_abnormal_train_features='C:/Users/Administrator/Desktop/GP Project/crimeucfdataset/Dataset/processed_c3d_features/processed_abnormal_train_features'
processed_test_features="C:/Users/Administrator/Desktop/GP Project/crimeucfdataset/Dataset/processed_c3d_features/processed_test_features"

normal_list = os.listdir(processed_normal_train_features)
normal_list.sort()
abnormal_list = os.listdir(processed_abnormal_train_features)
abnormal_list.sort()

In [ ]:
def save_model(model, json_path, weight_path):
    json_string = model.to_json()
    open(json_path, 'w').write(json_string)
    
    dict = {}
    i = 0
    for layer in model.layers:
        weights = layer.get_weights()
        my_list = np.zeros(len(weights), dtype=np.object)
        my_list[:] = weights
        dict[str(i)] = my_list
        i += 1
    scipy.io.savemat(weight_path, dict)

def load_model(json_path):
    model = model_from_json(open(json_path).read())
    return model

def load_batch_train(normal_path, normal_list, abnormal_path, abnormal_list):

    batchsize=60
    n_exp = int(batchsize/2)

    num_normal = len(normal_list)
    num_abnormal = len(abnormal_list)

    abnor_list_idx = np.random.permutation(num_abnormal)
    abnor_list = abnor_list_idx[:n_exp]
    norm_list_idx = np.random.permutation(num_normal)
    norm_list = norm_list_idx[:n_exp]

    abnormal_feats = []
    for video_idx in abnor_list:
        video_path = os.path.join(abnormal_path, abnormal_list[video_idx])
        with open(video_path, "rb") as f:
            feats = np.load(f)
        abnormal_feats.append(feats)

    normal_feats = []
    for video_idx in norm_list:
        video_path = os.path.join(normal_path, normal_list[video_idx])
        with open(video_path, "rb") as f:
            feats = np.load(f)
        normal_feats.append(feats)
        

    all_feats = np.vstack((*abnormal_feats, *normal_feats))
    all_labels = np.zeros(32*batchsize, dtype='uint8')

    all_labels[:32*n_exp] = 1

    return  all_feats, all_labels


def custom_objective(y_true, y_pred):

    y_true = K.reshape(y_true, [-1])
    y_pred = K.reshape(y_pred, [-1])
    n_seg = 32
    nvid = 60
    n_exp = int(nvid / 2)

    max_scores_list = []
    z_scores_list = []
    temporal_constrains_list = []
    sparsity_constrains_list = []

    for i in range(0, n_exp, 1):

        video_predictions = y_pred[i*n_seg:(i+1)*n_seg]

        max_scores_list.append(K.max(video_predictions))
        temporal_constrains_list.append(
            K.sum(K.pow(video_predictions[1:] - video_predictions[:-1], 2))
        )
        sparsity_constrains_list.append(K.sum(video_predictions))

    for j in range(n_exp, 2*n_exp, 1):

        video_predictions = y_pred[j*n_seg:(j+1)*n_seg]
        max_scores_list.append(K.max(video_predictions))

    max_scores = K.stack(max_scores_list)
    temporal_constrains = K.stack(temporal_constrains_list)
    sparsity_constrains = K.stack(sparsity_constrains_list)

    for ii in range(0, n_exp, 1):
        max_z = K.maximum(1 - max_scores[:n_exp] + max_scores[n_exp+ii], 0)
        z_scores_list.append(K.sum(max_z))

    z_scores = K.stack(z_scores_list)
    z = K.mean(z_scores)

    return z + \
        0.00008*K.sum(temporal_constrains) + \
        0.00008*K.sum(sparsity_constrains)


output_dir = trained_models
normal_dir = processed_normal_train_features
abnormal_dir = processed_abnormal_train_features

normal_list = os.listdir(normal_dir)
normal_list.sort()
abnormal_list = os.listdir(abnormal_dir)
abnormal_list.sort()

weights_path = output_dir + 'weights.mat'

model_path = output_dir + 'model.json'

#Create Full connected Model
model = classifier_model()

adagrad = tf.keras.optimizers.Adagrad(learning_rate=0.001, epsilon=1e-08)
adam = tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-08)

model.compile(loss=custom_objective, optimizer=adagrad)

if not os.path.exists(output_dir):
       os.makedirs(output_dir)

loss_graph =[]
num_iters = 25000
total_iterations = 0
batchsize=60
time_before = datetime.now()


for it_num in range(num_iters):
    inputs, targets = load_batch_train(
        normal_dir, normal_list, abnormal_dir, abnormal_list
    )
    batch_loss = model.train_on_batch(inputs, targets)
    loss_graph = np.hstack((loss_graph, batch_loss))
    total_iterations += 1
    if total_iterations % 20 == 0:
        print ("Iteration={} took: {}, loss: {}".format(
            total_iterations, datetime.now() - time_before, batch_loss)
        )
        #iteration_path = output_dir + 'Iterations_graph_' + str(total_iterations) + '.mat'
       # scipy.io.savemat(iteration_path, dict(loss_graph=loss_graph))
#     if total_iterations % 1000 == 0:  # Save the model at every 1000th iterations.
#         weights_path = output_dir + 'weightsAnomalyL1L2_' + str(total_iterations) + '.mat'
#         save_model(model, model_path, weights_path)

print("Train Successful - Model saved")
save_model(model, model_path, weights_path)

## Test Dataset Preparation

In [ ]:
#Test_Dataset training :

def load_test_set(videos_path, videos_list):
    feats = []
    
    for vid in videos_list:
        vid_path = os.path.join(videos_path, vid)
        with open(vid_path, "rb") as f:
            feat = np.load(f)
        feats.append(feat)

    feats = np.array(feats)
    return feats

classifier_model = build_classifier_model()

vid_list = os.listdir(processed_test_features)
vid_list.sort()

test_set = load_test_set(processed_test_features, vid_list)

for filename, example in zip(vid_list, test_set):
    predictions_file = filename[:-4] + '.npy'
    pred_path = os.path.join(preds_folder, predictions_file)
    pred = classifier_model.predict_on_batch(example)
    with open(pred_path, "wb") as f:
        np.save(pred_path, pred, allow_pickle=True)

## Results & Model Evaluation

In [ ]:

def eer_score(fpr, tpr, thr):
    """ Returns equal error rate (EER) and the corresponding threshold. """
    fnr = 1-tpr
    abs_diffs = np.abs(fpr - fnr)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((fpr[min_index], fnr[min_index]))
    return eer, thr[min_index]

ground_truth = pd.read_csv(
    test_temporal_annotations, header=None, index_col=0
)

preds = []
gts = []

for idx, row in ground_truth.iterrows():
    preds_file_path = os.path.join(preds_folder, idx)
    frames = row[6]
    try:
        with open(preds_file_path, "rb") as f:
            curr_preds = np.load(f)

        padded_preds = extrapolate(curr_preds, frames)
    except FileNotFoundError:
        padded_preds = np.zeros((frames,1))
        print("No predictions generated for {}".format(idx))

    curr_gts = np.zeros(frames)
    anomaly_start_1 =  row[2]
    anomaly_end_1 = row[3]

    anomaly_start_2 =  row[4]
    anomaly_end_2 = row[5]

    if anomaly_start_1 != -1 and anomaly_end_1 != -1:
        curr_gts[anomaly_start_1:anomaly_end_1+1] = 1

    if anomaly_start_2 != -1 and anomaly_end_2 != -1:
        curr_gts[anomaly_start_2:anomaly_end_2+1] = 1

    preds.append(padded_preds)
    gts.append(curr_gts)

gts = np.concatenate(gts)
preds = np.concatenate(preds)
preds_labels = np.round(preds)

acc = sklearn.metrics.accuracy_score(gts, preds_labels)
ap = sklearn.metrics.average_precision_score(gts, preds)
f1 = sklearn.metrics.f1_score(gts, preds_labels)
fpr, tpr, thr = sklearn.metrics.roc_curve(gts, preds)
prec, rec, _ = sklearn.metrics.precision_recall_curve(gts, preds)
eer, _ = eer_score(fpr, tpr, thr)
conf_mat = sklearn.metrics.confusion_matrix(gts, preds_labels)
auc = sklearn.metrics.auc(fpr, tpr)

plt.title("Curve ROC")
plt.plot(fpr, tpr, 'b', label = "AUC: {}".format(auc))
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig(os.path.join(output_folder, "roc.png"))

plt.clf()

plt.title("Curve Rec-Prec")
plt.plot(rec, prec, 'b', label = "Original - AP: {:.5f}".format(ap))
plt.legend(loc = 'lower right')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('Precison')
plt.xlabel('Recall')
plt.savefig(os.path.join(output_folder, "pr_curve.png"))
plt.show()

print("Accuracy: {:.5f}, AUC: {:.5f}, F1: {:.5f}, EER: {:.5f}, AP: {:.5F}".format(
    acc, auc, f1, eer, ap
))

print("Confusion matrix")
print(conf_mat)

## Show Results

In [ ]:
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation



def visualize_clip(clip, convert_bgr=False, save_gif=False, file_path=None):
    num_frames = len(clip)
    fig, ax = plt.subplots()
    fig.set_tight_layout(True)

    def update(i):
        if convert_bgr:
            frame = cv2.cvtColor(clip[i], cv2.COLOR_BGR2RGB)
        else:
            frame = clip[i]
        plt.imshow(frame)
        return plt

    # FuncAnimation will call the 'update' function for each frame; here
    # animating over 10 frames, with an interval of 20ms between frames.
    anim = FuncAnimation(fig, update, frames=np.arange(0, num_frames), interval=1)
    if save_gif:
        anim.save(file_path, dpi=80, writer='imagemagick')
    else:
        # plt.show() will just loop the animation forever.
        plt.show()


def visualize_predictions_full(video_path, predictions, save_path):
    frames = get_video_frames(video_path)
    assert len(frames) == len(predictions)
    anomaly=[]
    label=0
    prob=0
    for n in predictions:
        if n>=0.6:
            anomaly.append(n)
        if len(anomaly)>6:
            
            label='Anomaly'
            prob=np.mean(anomaly)
        else:
            label='Normal'
    fig, ax = plt.subplots(figsize=(5, 5))
    fig.set_tight_layout(True)

    fig_frame = plt.subplot(2, 1, 1)
    fig_prediction = plt.subplot(2, 1, 2)
    fig_prediction.set_xlim(0, len(frames))
    fig_prediction.set_ylim(0, 1.15)
    fig_prediction.set_title('{}:({:.4f})'.format(label, prob))
    
    
    def update(i):
        frame = frames[i]
        x = range(0, i)
        y = predictions[0:i]
        fig_prediction.plot(x, y, '-')
        fig_frame.imshow(frame)
        return plt

    # FuncAnimation will call the 'update' function for each frame; here
    # animating over 10 frames, with an interval of 20ms between frames.

    anim = FuncAnimation(fig, update, frames=np.arange(0, len(frames), 10), interval=1, repeat=False)

    if save_path:
           
        f=save_path
        writervideo = animation.FFMpegWriter(fps=16) 
        anim.save(f, writer=writervideo)
    else:
           plt.show()
   
    return anim
   

In [ ]:
output_folder='C:/Users/Administrator/Desktop/GP Project/output_folder_final'

In [ ]:
def run_demo():
    sample_video_path=r"C:\Users\Administrator\Desktop\GP Project\Test_split\test\Normal_Videos_941_x264.mp4"

    video_name = os.path.basename(sample_video_path).split('.')[0]

    # read video
    video_clips, num_frames = get_video_clips(sample_video_path)

    print("Number of clips in the video : ", len(video_clips))

    # build models
    feature_extractor = c3d_feature_extractor()
    classifier_model = build_classifier_model()

    print("Models initialized")

    # extract features
    rgb_features = []
    for i, clip in enumerate(video_clips):
        clip = np.array(clip)
        if len(clip) < frame_count:
            continue

        clip = preprocess_input(clip)
        rgb_feature = feature_extractor.predict(clip)[0]
        rgb_features.append(rgb_feature)

        print("Processed clip : ", i)

    rgb_features = np.array(rgb_features)
    rgb_feature_bag = interpolate(rgb_features, features_per_bag)
    
    # classify using the trained classifier model
    predictions = classifier_model.predict(rgb_feature_bag)

    predictions = np.array(predictions).squeeze()

    predictions = extrapolate(predictions, num_frames)
    
    save_path = os.path.join(output_folder,video_name+'.mp4').replace("\\",'/')
    # visualize predictions
    print('Executed Successfully - '+video_name + '.gif saved')
    visualize_predictions_full(sample_video_path, predictions, save_path)


if __name__ == '__main__':
    run_demo()